## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-14-09-00-00 +0000,cbc,Inuit leaders urge Ottawa to follow Greenland'...,https://www.cbc.ca/news/politics/inuit-leaders...
1,2026-02-14-09-00-00 +0000,cbc,Head of military's space division warns Russia...,https://www.cbc.ca/news/politics/russia-space-...
2,2026-02-14-09-00-00 +0000,cbc,From umbrella to uncertainty: Europe and Canad...,https://www.cbc.ca/news/politics/canada-france...
3,2026-02-14-08-47-21 +0000,nyt,Live Updates: Rubio Emphasizes Shared History ...,https://www.nytimes.com/live/2026/02/14/world/...
4,2026-02-14-08-42-07 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/14/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2508/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
109,trump,43
168,ice,19
119,epstein,19
161,new,15
58,house,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
131,2026-02-13-22-19-13 +0000,latimes,Newsom tells world leaders Trump's retreat on ...,https://www.latimes.com/california/story/2026-...,82
349,2026-02-13-10-00-16 +0000,wapo,Services cut. Staff laid off. New Trump cuts h...,https://www.washingtonpost.com/education/2026/...,78
145,2026-02-13-21-44-46 +0000,nypost,Ted Lieu makes wild claim about Trump and Epst...,https://nypost.com/2026/02/13/us-news/lieu-mak...,75
217,2026-02-13-18-33-12 +0000,nypost,AOC downplays 2028 talk in Munich — babbles wh...,https://nypost.com/2026/02/13/us-news/aoc-does...,74
51,2026-02-14-01-54-39 +0000,nyt,Trump Files Final Plans for White House Ballroom,https://www.nytimes.com/2026/02/13/us/politics...,73


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
131,82,2026-02-13-22-19-13 +0000,latimes,Newsom tells world leaders Trump's retreat on ...,https://www.latimes.com/california/story/2026-...
8,60,2026-02-14-06-47-49 +0000,nypost,White House unveils cheeky Valentine’s Day car...,https://nypost.com/2026/02/14/us-news/white-ho...
132,56,2026-02-13-22-13-46 +0000,nypost,Here’s the crude email exchange with Jeffrey E...,https://nypost.com/2026/02/13/business/heres-t...
232,53,2026-02-13-17-39-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
112,46,2026-02-13-22-41-16 +0000,startribune,DHS says ICE agents suspended for ‘untruthful’...,https://www.startribune.com/dhs-says-ice-agent...
164,44,2026-02-13-20-53-59 +0000,bbc,Ex-CNN host Don Lemon pleads not guilty in Min...,https://www.bbc.com/news/articles/cge89r1e158o...
106,41,2026-02-13-22-59-00 +0000,cbc,Labelling Palestine Action a terrorist group w...,https://www.cbc.ca/radio/asithappens/palestine...
36,41,2026-02-14-02-48-57 +0000,nypost,Latest Nancy Guthrie ransom note sent to TMZ i...,https://nypost.com/2026/02/13/us-news/new-nanc...
237,40,2026-02-13-17-11-14 +0000,latimes,San Francisco teachers' strike ends as union a...,https://www.latimes.com/california/story/2026-...
3,33,2026-02-14-08-47-21 +0000,nyt,Live Updates: Rubio Emphasizes Shared History ...,https://www.nytimes.com/live/2026/02/14/world/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
